In [1]:
import pandas as pd
import random
import numpy as np
import os

SEED = 123

In [2]:
outcome_df = pd.read_csv('/home/ngsci/datasets/brca-psj-path/v2/outcomes.csv')
map_df = pd.read_csv('/home/ngsci/datasets/brca-psj-path/v2/slide-biopsy-map.csv')
#Unique labels ['IA', 'IIB', 'IIA', '0', nan, 'IIIC', 'IV', 'IIIA', 'IIIB', 'IB']
label_mapping = {'IA':1, 'IB':1, 'IIA':2, 'IIB':2, 'IIIA':3, 'IIIB':3, 'IIIC':3, 'IV':4, '0':0}
outcome_df['label'] = outcome_df['stage'].map(label_mapping)
outcome_df = outcome_df[~pd.isna(outcome_df['label'])]

In [8]:
#stratified_df = outcome_df.groupby('label', group_keys=False).apply(lambda x: x.sample(min(len(x), 50)))
train_N = 600
train_df = outcome_df.groupby('label', group_keys=False).apply(lambda x: x.sample(int(np.rint(train_N*len(x)/len(outcome_df))))).sample(frac=1, random_state=SEED).reset_index(drop=True)

test_N = 600
remained_df = outcome_df[~outcome_df['biopsy_id'].isin(train_df['biopsy_id'])]
test_df = remained_df.groupby('label', group_keys=False).apply(lambda x: x.sample(int(np.rint(test_N*len(x)/len(remained_df))))).sample(frac=1, random_state=SEED).reset_index(drop=True)

train_mapping = map_df[map_df['biopsy_id'].isin(train_df['biopsy_id'])]
test_mapping = map_df[map_df['biopsy_id'].isin(test_df['biopsy_id'])]
print('Train slides: %d\n'%len(train_mapping))
print('Test slides: %d\n'%len(test_mapping))

path_prefix = '/home/ngsci/datasets/brca-psj-path/basic-downsampling/v2/'
train_mapping['downsampled_path'] = path_prefix + train_mapping['slide_id'] + '.png'
test_mapping['downsampled_path'] = path_prefix + test_mapping['slide_id'] + '.png'

Train slides: 10206

Test slides: 10205



/tmp/ipykernel_211/1558716067.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_mapping['downsampled_path'] = path_prefix + train_mapping['slide_id'] + '.png'
/tmp/ipykernel_211/1558716067.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_mapping['downsampled_path'] = path_prefix + test_mapping['slide_id'] + '.png'


In [9]:
check_res = train_mapping['downsampled_path'].map(lambda x: os.path.isfile(x))
assert(check_res.sum() == len(check_res))

check_res = test_mapping['downsampled_path'].map(lambda x: os.path.isfile(x))
assert(check_res.sum() == len(check_res))

In [10]:
train_df.to_csv('./csv_dir/train_outcomes.csv', index=False)
test_df.to_csv('./csv_dir/test_outcomes.csv', index=False)
train_mapping.to_csv('./csv_dir/train_mapping.csv', index=False)
test_mapping.to_csv('./csv_dir/test_mapping.csv', index=False)

In [12]:
holdout_df = pd.read_csv('/home/ngsci/datasets/brca-psj-path/holdout/v2/biopsy-info.csv')
holdout_mapping = pd.read_csv('/home/ngsci/datasets/brca-psj-path/holdout/v2/slide-biopsy-map.csv')

path_prefix = '/home/ngsci/datasets/brca-psj-path/basic-downsampling/holdout/'
holdout_mapping['downsampled_path'] = path_prefix + holdout_mapping['slide_id'] + '.png'


check_res = holdout_mapping['downsampled_path'].map(lambda x: os.path.isfile(x))
assert(check_res.sum() == len(check_res))

In [13]:
holdout_df.to_csv('./csv_dir/holdout_outcomes.csv', index=False)
holdout_mapping.to_csv('./csv_dir/holdout_mapping.csv', index=False)